In [1]:
from blockchain.lib.data.historical import *
from blockchain.lib.model import Seq2Seq
from blockchain.lib.data.dataset import Dataset
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
from blockchain.lib.data.historical import *
import pickle as pkl

In [2]:
pg = PoliniexGetter(start_date="2016-07-01")
btc = pg.get(crypto_name="USDT_BTC", market="poliniex")
eth = pg.get(crypto_name="USDT_ETH", market="poliniex")

In [3]:
eth["eth"]=eth["price"]
eth["eth_volume"]=eth["volume"]
dataset = pd.concat([btc, eth[["eth", "eth_volume"]]], axis=1, join="inner")

In [21]:
model = ["seq2seq_hidden",
         "seq2seq_eth_do_hidden",
         "6hours_2hidden_50unit_lenseq12_ywindow8",
         "6hours_1hidden_50unit_lenseq12_ywindow8",
         "only_price_6hours_2hidden_20unit_lenseq12_ywindow8"]

experiment=model[-1]

config = {
    "log_dir": './seq2seq/' + experiment,
    "len_sequence": 12,
    "rnn_layers": [{'num_units': 20, 'keep_prob': 0.45}, {'num_units': 20, 'keep_prob': 0.45}],
    "training_steps": 100000,
    "print_step": 100,
    "batch_size": 100,
    "ratio_train_val_test": [0.1,0.2],
    "column": ["price", "eth"],
    "label": "eth",
    "early_stopping": 100000,
    "learning_rate": 1e-5,
    "optimizer": "RMSProp",
    "y_window": 8,
    "keep_prob": 0.45,
    "output_dim": 1,
    "GO_SYMBOL": 0.,
    "l2": 0.001,
    "bias_loss": 0,
    "experiment": experiment,
    "period": 6
}


seq2seq = Seq2Seq(config)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_num_ps_replicas': 0, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_session_config': None, '_save_checkpoints_secs': 600, '_model_dir': './seq2seq/only_price_6hours_2hidden_20unit_lenseq12_ywindow8', '_environment': 'local', '_save_summary_steps': 100, '_task_id': 0, '_evaluation_master': '', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6f2b124a20>, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_num_worker_replicas': 0, '_save_checkpoints_steps': None, '_log_step_count_steps': 100}


In [18]:
dataset = dataset.fillna(method="ffill")
dataset = dataset.dropna()
x, y = Dataset.create_series_dataset(dataset, config["column"],
                                             config["label"],
                                             config["len_sequence"],
                                             ratio=config["ratio_train_val_test"],
                                             y_window=config["y_window"],
                                             shuffle=True,
                                             period=config["period"])

pkl.dump([config, x, y], open("data_" + config["experiment"], "wb"))

/media/niki/DATA/private/blockchainanalysis/blockchain/lib/data/dataset/dataset.py:20: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [19]:
config, x, y = pkl.load(open("data_" + config["experiment"].replace("/", "_"), "rb"))

In [ ]:
seq2seq.fit(x, y)

Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./seq2seq/only_price_6hours_2hidden_20unit_lenseq12_ywindow8/model.ckpt-27520
INFO:tensorflow:Saving checkpoints for 27521 into ./seq2seq/only_price_6hours_2hidden_20unit_lenseq12_ywindow8/model.ckpt.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
INFO:tensorflow:S

In [10]:
b=0
loss = 0
r = 90
end = b + r
pred = seq2seq.predict(x["train"][b:end])
loss = pred - y["train"][b:end].reshape([r, config["y_window"]])
loss = np.sqrt(loss**2).mean()
loss, x["train"].shape

INFO:tensorflow:Restoring parameters from ./seq2seq/6hours_2hidden_50unit_lenseq12_ywindow8/model.ckpt-39798


(22.400936, (1264, 12, 4))

In [ ]:
import matplotlib.pyplot as plt
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
from blockchain.lib.data.historical import *
prediction=np.hstack([np.array([np.array([np.nan for j in range(config["len_sequence"])]) for i in range(0,100,config["len_sequence"]+config["y_window"])]), pred]).astype(np.float32).flatten()
pred.fill(np.nan)
nrm=np.hstack([np.array([np.array(sin["label"])[i:i+config["len_sequence"]] for i in range(0,100,config["len_sequence"]+config["y_window"])]), pred]).astype(np.float32).flatten()
sct=go.Scatter(x=sin["x"].index[:200], y=prediction)
sct2=go.Scatter(x=sin["x"].index[:200], y=nrm)
py.iplot([sct2, sct])

In [13]:
n=30
train_test="test"
pred = seq2seq.predict(x[train_test][b:end]) 
prediction=np.hstack([np.array([np.nan for j in range(config["len_sequence"])]), pred[n]]).astype(np.float32).flatten()
nrm=x[train_test][n][:,1].astype(np.float32).flatten()
trh=np.hstack([np.array([np.nan for j in range(config["len_sequence"])]), y[train_test][n].astype(np.float32).flatten()]).astype(np.float32).flatten()
sct=go.Scatter(x=[i for i in range(prediction.shape[0])], y=prediction)
sct2=go.Scatter(x=[i for i in range(prediction.shape[0])], y=nrm)
sct3=go.Scatter(x=[i for i in range(prediction.shape[0])], y=trh)
py.iplot([sct2, sct, sct3])

INFO:tensorflow:Restoring parameters from ./seq2seq/6hours_2hidden_50unit_lenseq12_ywindow8/model.ckpt-39798
